Use case: 12 linked windows with one candidate loaded for each.

In [ ]:
import os

from ipywidgets import Label, Button, HBox, VBox, Layout

from ginga.misc import Datasrc
from ginga.misc.log import get_logger
from ginga.util import wcsmod
from ginga.util.iohelper import get_fileinfo

from astrowidgets import ImageWidget

In [ ]:
# Reading ASDF in Ginga needs this to be specified early on
wcsmod.use('astropy_ape14')

In [ ]:
logger = get_logger('my viewer', log_stderr=True, log_file=None, level=30)

In [ ]:
class LinkedImageWidget(ImageWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        num_images = 10  # Max images in cache; should user be able to set this?
        self.datasrc = Datasrc.Datasrc(num_images)  # Cache
        self._other_viewers = []

    # Need jwst package.
    def load_jwst_asdf(self, filename):
        if filename in self.datasrc:
            image = self.datasrc[filename]
        else:
            import asdf
            from ginga.AstroImage import AstroImage

            image = AstroImage(logger=self.logger)
            image.load_file(filename, data_key='data')

            self.datasrc[filename] = image

        self._viewer.set_image(image)

    def load_fits(self, filename):
        bnch = get_fileinfo(filename)
        
        if filename in self.datasrc:
            image = self.datasrc[filename]
            self._viewer.set_image(image)
        else:
            super().load_fits(bnch.filepath, numhdu=bnch.numhdu)
            self.datasrc[filename] = self._viewer.get_image()
        
    def link_viewer(self, other_viewer):
        if other_viewer in self._other_viewers:
            raise ValueError('Viewer already linked')
        self._other_viewers.append(other_viewer)

    @property
    def n_linked_viewers(self):
        return len(self._other_viewers)

    def _mouse_click_cb(self, viewer, event, data_x, data_y):
        image = viewer.get_image()
        if image is None:  # Nothing to do
            return
        
        super()._mouse_click_cb(self._viewer, event, data_x, data_y)

        for ov in self._other_viewers:
            if not isinstance(ov, ImageWidget):
                continue
    
            other_image = ov._viewer.get_image()
            if other_image is None:
                continue
                
            if image.wcs.wcs is not None and other_image.wcs.wcs is not None:
                ra, dec = image.pixtoradec(data_x, data_y)
                other_x, other_y = other_image.radectopix(ra, dec)
            else:
                other_x, other_y = data_x, data_y

            ov._mouse_click_cb(ov._viewer, event, other_x, other_y)

In [ ]:
n_widgets = 12
w = []
w_labels = []

for _ in range(n_widgets):
    i_w = LinkedImageWidget(logger=logger)
    i_w.click_center = True
    w.append(i_w)
    w_labels.append(Label('Filename'))

In [ ]:
# First widget is primary viewer for WCS matching
for i in range(1, n_widgets):
    w[0].link_viewer(w[i])

In [ ]:
fits_files = [
    '/redkeep/ironthrone/ssb/stginga/test_data/jw42424001001_01101_00001_nrca5_assign_wcs.fits',
    '/redkeep/ironthrone/ssb/stginga/test_data/jw42424001001_01101_00002_nrca5_assign_wcs.fits',
    '/redkeep/ironthrone/ssb/stginga/test_data/jw42424001001_01101_00003_nrca5_assign_wcs.fits']
n_files = len(fits_files)

In [ ]:
# Assign data to widgets.
# If there is not enough data, repeat the data for testing purposes.
for i in range(n_widgets):
    filename = fits_files[i % n_files]
    # w[i].load_fits(filename)  # FITS
    w[i].load_jwst_asdf(filename)  # JWST ASDF-in-FITS
    w_labels[i].value = os.path.basename(filename)

In [ ]:
button_zoomlevel = Button(description="Sync zoom level")

def on_button_zoomlevel_clicked(b):
    for i in range(1, n_widgets):
        w[i].zoom_level = w[0].zoom_level

button_zoomlevel.on_click(on_button_zoomlevel_clicked)

In [ ]:
# Manual arrangement for now
VBox([button_zoomlevel,
      HBox([VBox([w_labels[0], w[0]], layout=Layout(margin='0 10px 0 0')),
            VBox([w_labels[1], w[1]], layout=Layout(margin='0 10px 0 0')),
            VBox([w_labels[2], w[2]])]),
      HBox([VBox([w_labels[3], w[3]], layout=Layout(margin='0 10px 0 0')),
            VBox([w_labels[4], w[4]], layout=Layout(margin='0 10px 0 0')),
            VBox([w_labels[5], w[5]])]),
      HBox([VBox([w_labels[6], w[6]], layout=Layout(margin='0 10px 0 0')),
            VBox([w_labels[7], w[7]], layout=Layout(margin='0 10px 0 0')),
            VBox([w_labels[8], w[8]])]),
      HBox([VBox([w_labels[9], w[9]], layout=Layout(margin='0 10px 0 0')),
            VBox([w_labels[10], w[10]], layout=Layout(margin='0 10px 0 0')),
            VBox([w_labels[11], w[11]])])])

#### Instructions

Images are pre-loaded into viewers already.

Click on the top left viewer and the rest will match in RA and Dec of centered point. No distortion correction is performed at this point.

Press `+`/`-` to zoom in/out. Click "Sync zoom level" button to have the viewers match the zoom level of the top left one.